In [58]:
from math import ceil, sqrt
import numpy as np

In [77]:
import logging
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(message)s') # Additional args: %(asctime)s %(levelname)s
log.setLevel(logging.DEBUG) # Comment or modify to INFO to avoid verbose logs

In [78]:
# Storage mass constant
C = 10**12
# Number of dworks per 1 KAS (dworks are also known as sompis, and are the smallest KAS unit)
S = 10**8
# Maximal (standard) mass per Tx
M = 10**5
# Current standard mass-to-fee ratio is 1, meaning that a Tx with M mass is expected to pay a fee of M dworks 
MASS_FEE_RATIO = 1
# Standard mass for a Tx with maximal mass 
F = MASS_FEE_RATIO * M
# The threshold for which below it we give up the change and turn it into fee
CHANGE_THRESHOLD = F
# The maximum storage mass allowed to use for a single payment (which could be composed of several chained txs)
MAX_PAYMENT_MASS = 20 * M

In [4]:
SPK_STD_LEN = 36 # version len + p2pk len
SIG_STD_LEN = 66 # schnorr sig is 64 bytes + 2 op data bytes
OUT_SIZE = 8 + 2 + 8 + SPK_STD_LEN
IN_SIZE = 32 + 4 + 8 + SIG_STD_LEN + 8
TX_FIELDS_SIZE = 2 + 8 + 8 + 8 + 20 + 8 + 32 + 8

def estimated_tx_size(inputs_num, outputs_num):
    return TX_FIELDS_SIZE + inputs_num * IN_SIZE + outputs_num * OUT_SIZE 

"""
Calculation of transaction compute mass. For simplicity, the calculation assumes
standard signatures and scripts. However other than that it is accurate.
"""
def compute_mass(inputs, outputs):
    inputs_num, outputs_num = len(inputs), len(outputs)
    return 1000 * inputs_num + 10 * SPK_STD_LEN * outputs_num + estimated_tx_size(inputs_num, outputs_num)

In [5]:
"""
Calculates the negative component of the storage mass formula. Note that there is no dependency on output
values but only on their count. The code is designed to maximize precision and to avoid intermediate overflows.
"""
def negative_mass(inputs, outputs_num):
    inputs_num = len(inputs)
    if outputs_num == 1 or inputs_num == 1 or (outputs_num == 2 and inputs_num == 2):
        return sum(C//v for v in inputs)
    return inputs_num*(C//(sum(inputs)//inputs_num)) 

"""
Calculates the storage mass for the provided input/output collections 
"""
def storage_mass(inputs, outputs):
    N = negative_mass(inputs, outputs_num=len(outputs))
    P = sum(C//o for o in outputs)
    return max(P-N, 0)

"""
Calculates the overall mass of this transaction
"""
def mass(inputs, outputs):
    return max(storage_mass(inputs, outputs), compute_mass(inputs, outputs)) 

In [80]:
"""
Implements the algorithm for creating a minimal chain of transactions which can make 
the desired payment without passing storage mass limits locally per transaction
"""
def create_payment_chain(inputs, payment):
    txs = []
    while storage_mass(inputs, outputs=[payment, sum(inputs) - payment - F]) > M:
        T = sum(inputs) - F
        N = negative_mass(inputs, 2) 
        D = (M + N)*T/C
        if D**2 - 4*D < 0:
            # Indiactes that a single small input was used. Adding another input might help 
            raise Exception('The input is too small, try adding one more input (negative sqrt)')
        # Single step optimization, taking the smaller solution
        alpha = (D - sqrt(D**2 - 4*D))/(2*D) 
        # Round *up* in order to not increase the mass above M
        outputs = [ceil(alpha * T), T - ceil(alpha * T)]
        if inputs == outputs:
            # The process reached a point where the ceil round up prevents further progress.
            raise Exception('The inputs and the payment need to be closer in value (integer precision)')
        txs.append((inputs, outputs))
        log.debug(storage_mass(inputs, outputs))
        inputs = outputs
    txs.append((inputs, [payment, sum(inputs) - payment - F]))
    log.debug(storage_mass(*txs[-1]))
    return txs

In [96]:
"""
Helper class for managing the available UTXO entries
"""
class Context:
    def __init__(self, utxos):
        # The full list of currently available UTXO entries
        self.utxos = utxos.copy()
        log.debug(self.utxos)
    
    def has_any(self):
        return len(self.utxos) > 0
    
    def has_above(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        return any(utxos >= threshold)
    
    def pop_min_above(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        large = utxos[utxos >= threshold]
        if len(large) > 0:
            item = min(large)
        else:
            item = max(utxos)
        self.utxos.remove(item)
        return item

    def pop_max_below(self, threshold):
        utxos = np.array(self.utxos, dtype=np.uint64)
        small = utxos[utxos <= threshold]
        if len(small) > 0:
            item = max(small)
        else:
            item = min(utxos)
        self.utxos.remove(item)
        return item
    
    def pop_max(self):
        item = max(self.utxos)
        self.utxos.remove(item)
        return item

    def pop_min(self):
        item = min(self.utxos)
        self.utxos.remove(item)
        return item
    

In [95]:
"""
Main entry point for a send operation. Using the available `utxos`, creates transaction(s)
for sending the desired `amount`, while minimizing mass and fees as much as possible
"""
def create_transactions(utxos, amount):
    
    # Baseline mass for a 1:1 Tx
    min_mass = compute_mass([amount], [amount])
    # Baseline fee
    min_fee = min_mass * MASS_FEE_RATIO
    
    if sum(utxos) < amount + min_fee:
        raise Exception('insufficient funds')
    
    if sum(utxos) - amount - min_fee < CHANGE_THRESHOLD:
        # If the change value is too small, the storage mass will be large. At some range
        # it is best to simply give it up and add it to the fee (of course this can be 
        # reconfigured based on user feedback) 
        return [(utxos, [amount])] # TODO: compounding
        
        
    # Baseline mass for a 1:2 Tx
    min_mass = compute_mass([amount], [amount//2, amount//2])
    # Baseline fee
    min_fee = min_mass * MASS_FEE_RATIO
    
    # Processing context
    ctx = Context(utxos)
    
    if ctx.has_above(threshold=amount+min_fee):
        # Storage mass zone
        inputs = [ctx.pop_min_above(threshold=amount+min_fee)] # Take the minimal utxo with sufficient funds
        change = sum(inputs) - amount - min_fee
        if change == 0:
            return [(inputs, [amount])]
        storm = storage_mass(inputs, [amount, change])
        if storm > min_mass and ctx.has_any():
            # Adding another input will reduce the storage mass 
            inputs.append(ctx.pop_max_below(amount-min_fee))
        return create_payment_chain(inputs, payment=amount)
            
    else:
        # Compute mass zone -- all utxos are smaller than `amount`
        # In this case all current wallet algorithms should work without
        # modification, including compounding logic etc. The only exception
        # is the case where the change is randomly small which should be
        # handled either by adding inputs or by giving up the change and adding 
        # it as fee
        raise Exception('not implemented - storage mass logic is insignificant for compounding cases')

create_transactions([10*S, 2*S, 12*S, 3*S], 11*S)

[1000000000, 200000000, 1200000000, 300000000]
0


[([1200000000, 1000000000], [1100000000, 1099900000.0])]